# Validate Analysis Results

This notebook compares analysis outputs between **production runs** (baseline) and **test runs** to validate that tests produced expected results.

**Modes:**
- **Single validation**: Compare one production/test pair for one perspective (Section 3A)
- **Batch validation**: Compare multiple pairs from CSV/XLSX, all perspectives (Section 3B)

**Perspectives validated (batch mode):** GR (Gross), GU (Ground-Up), RL (Reinsurance Layer)

**Endpoints compared:**
- Settings (analysis configuration)
- Statistics (`/stats`)
- EP Metrics (`/ep`)
- Event Loss Table (`/elt`)
- Period Loss Table (`/plt`) - HD analyses only

**Fields compared per endpoint:**

| Endpoint | Fields |
|----------|--------|
| Settings | engineType, engineVersion, analysisType, analysisMode, analysisFramework, modelProfile, outputProfile, eventRateSchemeNames, peril, perilCode, subPeril, region, regionCode, lossAmplification, insuranceType, vulnerabilityCurve, engineSubType, isMultiEvent |
| Statistics | epType, purePremium, totalStdDev, cv, netPurePremium, activation, exhaustion, totalLossRatio, limit, premium, netStdDev, exhaustAllReinstatements |
| EP Metrics | epType, value (contains returnPeriods and positionValues arrays) |
| ELT | eventId, sourceId, positionValue, stdDevI, stdDevC, expValue, rate, peril, region, oepWUC |
| PLT | periodId, eventId, weight, eventDate, lossDate, positionValue, peril, region |

## 1. Setup & Imports

In [1]:
%load_ext autoreload
%autoreload 2

from helpers.analysis_results_validator import (
    AnalysisResultsValidator,
    # Single validation output
    print_validation_summary,
    print_validation_details,
    # Batch validation output
    batch_results_to_dataframe,
    print_batch_summary,
    export_batch_failures_to_json,
    export_batch_summary_to_csv,
)

validator = AnalysisResultsValidator()
print("Setup complete.")

Setup complete.


## 2. Configuration

**Common settings** apply to both single and batch validation.

In [2]:
# === Common Settings ===

# Comparison settings
RELATIVE_TOLERANCE = 1e-4  # For floating-point comparison
MAX_DIFFERENCES_TO_SHOW = 50  # Limit output for large datasets

# Note: PLT comparison is automatically included for HD analyses only (based on engineType)

## 3A. Single Validation (One Pair, One Perspective)

Use this section to validate a single production/test pair for a specific perspective.

Enter the **appAnalysisId** values - these are the IDs shown in the Moody's RiskModeler UI (e.g., 35810).

In [3]:
# Single validation configuration
PRODUCTION_APP_ANALYSIS_ID = 1649  # Replace with your production analysis ID
TEST_APP_ANALYSIS_ID = 4425        # Replace with your test analysis ID
PERSPECTIVE_CODE = 'GR'            # 'GR' (Gross), 'GU' (Ground-Up), 'RL' (Reinsurance Layer)

# Run single validation
result = validator.validate(
    production_app_analysis_id=PRODUCTION_APP_ANALYSIS_ID,
    test_app_analysis_id=TEST_APP_ANALYSIS_ID,
    perspective_code=PERSPECTIVE_CODE,
    relative_tolerance=RELATIVE_TOLERANCE,
)

print_validation_summary(result)

ANALYSIS VALIDATION RESULTS

Production Analysis ID: 1649
Test Analysis ID:       4425
Perspective:            GR

------------------------------------------------------------
Endpoint Results:
------------------------------------------------------------
  [OK] Settings: PASS
  [X] Statistics: FAIL (Error: Record count mismatch: prod=1, test=0)
  [X] EP Metrics: FAIL (Error: Record count mismatch: prod=4, test=0)
  [X] ELT: FAIL (46 value differences)

OVERALL: FAIL


### Single Validation Details (if failed)

In [4]:
print_validation_details(result, max_differences=MAX_DIFFERENCES_TO_SHOW)


Statistics DIFFERENCES

Error: Record count mismatch: prod=1, test=0

EP Metrics DIFFERENCES

Error: Record count mismatch: prod=4, test=0

ELT DIFFERENCES

Value differences (46 records with differences):

  Key: 1510016
    rate:
      prod: 0.182068552
      test: 0.0
    oepWUC:
      prod: 0.7130690613326276
      test: 0.0

  Key: 1510017
    rate:
      prod: 0.267865285
      test: 0.0
    oepWUC:
      prod: 0.8319250383837815
      test: 0.0

  Key: 1510018
    rate:
      prod: 0.218801909
      test: 0.0
    oepWUC:
      prod: 0.999994605268582
      test: 0.0

  Key: 1510019
    rate:
      prod: 0.142006308
      test: 0.0
    oepWUC:
      prod: 0.996293985583191
      test: 0.0

  Key: 1510020
    rate:
      prod: 0.210022789
      test: 0.0
    oepWUC:
      prod: 0.999993285799756
      test: 0.0

  Key: 1510021
    rate:
      prod: 0.118732103
      test: 0.0
    oepWUC:
      prod: 0.9987117512061159
      test: 0.0

  Key: 1510022
    rate:
      prod: 0.143739

## 3B. Batch Validation (Multiple Pairs, All Perspectives)

Use this section to validate multiple production/test pairs from a CSV or XLSX file.
All three perspectives (GR, GU, RL) are validated automatically.

**File format (CSV or XLSX):**
```
production_app_analysis_id,test_app_analysis_id,name
1575,4342,USFL_Other
1576,4343,USEQ_Primary
1577,4344,USHU_Full
```

Required columns:
- `production_app_analysis_id`: App analysis ID for production (baseline)
- `test_app_analysis_id`: App analysis ID for test run
- `name` (optional): Label for this analysis pair

In [6]:
# Path to CSV or XLSX file with analysis pairs
FILE_PATH = 'analysis_results/Analysis Results.xlsx'  # Replace with your file path (.csv or .xlsx)

# Progress callback (optional)
def show_progress(current, total, name, perspective):
    print(f"[{current}/{total}] {name} - {perspective}")

# Run batch validation (validates all perspectives: GR, GU, RL)
# PLT is automatically included for HD analyses only
batch_result = validator.validate_batch_from_file(
    file_path=FILE_PATH,
    relative_tolerance=RELATIVE_TOLERANCE,
    progress_callback=show_progress,
)

print()
print_batch_summary(batch_result)

Resolving analysis IDs for 40 analysis pairs...
[1/40] USFF_Geico_75HIP2 - Looking up
[2/40] USFF_Condo - Looking up
[3/40] USFF_Lender_P - Looking up
[4/40] USFL_Other_Other_LT - Looking up
[5/40] USFL_Other_Renters_LT - Looking up
[6/40] USFL_Other_Manufact_LT - Looking up
[7/40] USFL_Other_Lender_P_LT - Looking up
[8/40] USFL_Excess_LT - Looking up
[9/40] USFL_Commercial_LT - Looking up
[10/40] USHU_Full_Clay_Homes_LT - Looking up
[11/40] USHU_Full_Clay_21st_LT - Looking up
[12/40] USHU_Full_Lender_P_LT - Looking up
[13/40] USHU_Leak_Clay_Homes_LT - Looking up
[14/40] USHU_Leak_Clay_21st_LT - Looking up
[15/40] USHU_Leak_Lender_P_LT - Looking up
[16/40] USOW_Clay_Homes_High_Frq - Looking up
[17/40] USOW_Clay_21st_High_Frq - Looking up
[18/40] USOW_Other_High_Frq - Looking up
[19/40] USOW_Geico_75HIP2_High_Frq - Looking up
[20/40] USWF_Clay_Homes - Looking up
[21/40] USWF_Clay_21st - Looking up
[22/40] USWF_Other - Looking up
[23/40] USWF_CHFS - Looking up
[24/40] USWF_Geico_75HIP2 -

### Batch Results Summary Table

In [4]:
# Display results as a DataFrame
df = batch_results_to_dataframe(batch_result)
df

""


### Export Results (Optional)

Export the summary to CSV and detailed failures to JSON.

In [ ]:
# Export summary to CSV (saved to validation_outputs folder)
summary_path = export_batch_summary_to_csv(batch_result)
print(f"Summary exported to: {summary_path}")

# Export detailed failures to JSON (only if there are failures)
if batch_result.failed_count > 0:
    failures_path = export_batch_failures_to_json(
        batch_result,
        max_differences=MAX_DIFFERENCES_TO_SHOW
    )
    print(f"Failure details exported to: {failures_path}")
else:
    print("No failures to export.")